<a href="https://colab.research.google.com/github/kmvenkatesh04/Data-Science-Engg/blob/master/SEM2/ACI/Bayesian_Model_Aswin_Cricket_Analysis_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""

column = list(AttribInp.keys())
condition = list(AttribInp.values())
equal = ['==' for i in condition]
query = ' & '.join(f'{i} {j} {repr(k)}' for i, j, k in zip(column, equal, condition))

"""

Nodes for Bayesian network:

*   Test Location (Home, Away)
*   Aswin Playing (Yes, No)
*   Toss (Win, Loss, Draw)
*   Batting (1st, 2nd)
*   Result (Win, Loss, Draw)


In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive') """

Install Pomogranate module

In [ ]:
#!pip install pomegranate

Importing necessary modules

In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pomegranate import *

sns.set_style('whitegrid')
%matplotlib inline


Load input data file and get the file info & attributes

In [241]:
AswnIndTestData = pd.read_excel('/content/drive/My Drive/ColabDataInputs/India_Test_Stats.xlsx',sheet_name='Sheet1')
print('The shape of the data is ',AswnIndTestData.shape)
print('The details of the data set are \n')
print(AswnIndTestData.info())

The shape of the data is  (85, 8)
The details of the data set are 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Ashwin      85 non-null     object        
 1   Result      85 non-null     object        
 2   Margin      85 non-null     object        
 3   Toss        85 non-null     object        
 4   Bat         85 non-null     object        
 5   Opposition  85 non-null     object        
 6   Location    85 non-null     object        
 7   Start Date  85 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 5.4+ KB
None


In [242]:
AswnIndTestData[AswnIndTestData.Opposition == 'v Australia'].head()

,Ashwin,Result,Margin,Toss,Bat,Opposition,Location,Start Date
3,Y,lost,122 runs,lost,2nd,v Australia,Away,2011-12-26
4,Y,lost,inns & 68 runs,won,1st,v Australia,Away,2012-01-03
5,N,lost,inns & 37 runs,lost,1st,v Australia,Away,2012-01-13
6,Y,lost,298 runs,lost,2nd,v Australia,Away,2012-01-24
13,Y,won,8 wickets,lost,2nd,v Australia,Home,2013-02-22


Changing the attribute names in the below fashion

*   Test Location - TLoc
*   Aswin Playing (Yes, No) - Aswin
*   Toss (Win, Loss, Draw) - Toss
*   Batting (1st, 2nd) - Bat
*   Result (Win, Loss, Draw) - Result

In [243]:
AttribChose = ['Ashwin', 'Result', 'Location', 'Toss', 'Bat']
AttribVals = {i:list(AswnIndTestData[i].unique()) for i in AttribChose}
AttribInp = {}

def getInput(attribval,attrib):
  print('**************************************************************\n')
  print('Choose any of the values {} for {}'.format(attribval,attrib))
  InputVal = input('')
  return InputVal

"""
for i in range(len(AttribChose)):
  a = getInput(AttribVals[AttribChose[i]],AttribChose[i])
  while a not in AttribVals[AttribChose[i]]:
    print('You have entered an incorrect option. \nYou have entered {}.\nPlease check case and spelling and try againui'.format(a))
    a = getInput(AttribVals[AttribChose[i]],AttribChose[i])
  AttribInp[AttribChose[i]] = a

"""

"\nfor i in range(len(AttribChose)):\n  a = getInput(AttribVals[AttribChose[i]],AttribChose[i])\n  while a not in AttribVals[AttribChose[i]]:\n    print('You have entered an incorrect option. \nYou have entered {}.\nPlease check case and spelling and try againui'.format(a))\n    a = getInput(AttribVals[AttribChose[i]],AttribChose[i])\n  AttribInp[AttribChose[i]] = a\n\n"

In [244]:
def getDiscreteProba(df,attrib):
  attribcount = df[attrib].value_counts().to_dict()
  attribproba = {i:attribcount[i]/sum(attribcount.values()) for i in attribcount.keys()}
  return attribcount, attribproba


def filterDF( InputVals , df):
  column = list(InputVals.keys())
  condition = list(InputVals.values())
  equal = ['==' for i in condition]
  query = ' & '.join(f'{i} {j} {repr(k)}' for i, j, k in zip(column, equal, condition))
  filterdf = df.query(query)
  return filterdf

def getWinProbab(inputvalues,df):
  tempDF = filterDF(inputvalues, df)
  if len(tempDF) > 0:
    tempDFPvt = pd.pivot_table(tempDF,index=["Result"],values=[list(inputvalues.keys())[0]],aggfunc='count')
    tempDFPvt.reset_index(inplace = True)
    tempDFPvt.rename(columns={list(inputvalues.keys())[0]:'Count'},inplace = True)
    WinProbab = {i:int(tempDFPvt[tempDFPvt.Result == i].Count)/sum(list(tempDFPvt['Count'])) for i in list(tempDFPvt['Result'])}
    #print(WinProbab)
    try:
      condproblist = list(inputvalues.keys())
      condproblist.append(WinProbab['won'])
    except:
      condproblist = list(inputvalues.keys())
      condproblist.append(float(0))
      #print(condproblist)
  else:
    print('The condition is invalid. No values matching the condition ',(inputvalues))
    condproblist = list(inputvalues.keys())
    condproblist.append(float(0)) 
    WinProbab,tempDFPvt = {'draw': 0.0, 'lost': 0.0, 'won': 0.0},pd.DataFrame()
  return condproblist,WinProbab,tempDFPvt

In [245]:
AttribMap = {'TLoc':'Location','Aswin':'Ashwin','Toss':'Toss','Bat':'Bat','Result':'Result'}
DiscreteProbDict = {list(AttribMap.keys())[i]:getDiscreteProba(AswnIndTestData,list(AttribMap.values())[i])[1] for i in range(len(list(AttribMap.values())))}

In [246]:
df = AswnIndTestData

In [167]:
combs[0]

{'Ashwin': 'Y', 'Bat': '2nd', 'Location': 'Home', 'Toss': 'lost'}

In [283]:
keys = ['Ashwin', 'Location', 'Toss', 'Bat']
combs = []
for i in AttribVals[keys[0]]:
  a = {}
  a[keys[0]] = i
  for j in AttribVals[keys[1]]:
    b = {}
    b[keys[1]] = j
    b.update(a)
    for k in AttribVals[keys[2]]:
      c = {}
      c[keys[2]] = k
      c.update(b)
      for l in AttribVals[keys[3]]:
        d = {}
        d[keys[3]] = l
        d.update(c)
        combs.append(d)


Condprob = []
for i in combs:
  #print(i)
  #print(getWinProbab(i, AswnIndTestData)[0])
  Condprob.append(getWinProbab(i, df)[0])

The condition is invalid. No values matching the condition  {'Bat': '2nd', 'Toss': 'won', 'Location': 'Away', 'Ashwin': 'Y'}
The condition is invalid. No values matching the condition  {'Bat': '2nd', 'Toss': 'lost', 'Location': 'Home', 'Ashwin': 'N'}
The condition is invalid. No values matching the condition  {'Bat': '1st', 'Toss': 'lost', 'Location': 'Home', 'Ashwin': 'N'}
The condition is invalid. No values matching the condition  {'Bat': '2nd', 'Toss': 'won', 'Location': 'Home', 'Ashwin': 'N'}
The condition is invalid. No values matching the condition  {'Bat': '1st', 'Toss': 'won', 'Location': 'Home', 'Ashwin': 'N'}


In [306]:
TLoc = DiscreteDistribution( DiscreteProbDict['TLoc'] )
Aswin = DiscreteDistribution( DiscreteProbDict['Aswin'])
Toss = DiscreteDistribution( DiscreteProbDict['Toss'])
Bat = DiscreteDistribution( DiscreteProbDict['Bat'])
Result = ConditionalProbabilityTable(Condprob,[Bat])

In [285]:
DiscreteProbDict['TLoc']

{'Away': 0.49411764705882355, 'Home': 0.5058823529411764}

In [307]:
s1 = Node(TLoc, name="Location")
s2 = Node(Aswin, name="Ashwin Playing")
s3 = Node(Toss, name="Toss")
s4 = Node(Bat, name="Batting")
s5 = Node(Result, name="Result")

In [308]:
model = BayesianNetwork("Winning probablity of Aswin Playing in the game")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s3, s4)
model.add_edge(s4, s5)


In [290]:
combs

[{'Ashwin': 'Y', 'Bat': '2nd', 'Location': 'Home', 'Toss': 'lost'},
 {'Ashwin': 'Y', 'Bat': '1st', 'Location': 'Home', 'Toss': 'lost'},
 {'Ashwin': 'Y', 'Bat': '2nd', 'Location': 'Home', 'Toss': 'won'},
 {'Ashwin': 'Y', 'Bat': '1st', 'Location': 'Home', 'Toss': 'won'},
 {'Ashwin': 'Y', 'Bat': '2nd', 'Location': 'Away', 'Toss': 'lost'},
 {'Ashwin': 'Y', 'Bat': '1st', 'Location': 'Away', 'Toss': 'lost'},
 {'Ashwin': 'Y', 'Bat': '2nd', 'Location': 'Away', 'Toss': 'won'},
 {'Ashwin': 'Y', 'Bat': '1st', 'Location': 'Away', 'Toss': 'won'},
 {'Ashwin': 'N', 'Bat': '2nd', 'Location': 'Home', 'Toss': 'lost'},
 {'Ashwin': 'N', 'Bat': '1st', 'Location': 'Home', 'Toss': 'lost'},
 {'Ashwin': 'N', 'Bat': '2nd', 'Location': 'Home', 'Toss': 'won'},
 {'Ashwin': 'N', 'Bat': '1st', 'Location': 'Home', 'Toss': 'won'},
 {'Ashwin': 'N', 'Bat': '2nd', 'Location': 'Away', 'Toss': 'lost'},
 {'Ashwin': 'N', 'Bat': '1st', 'Location': 'Away', 'Toss': 'lost'},
 {'Ashwin': 'N', 'Bat': '2nd', 'Location': 'Away', 'To

In [309]:
model.bake()

IndexError: ignored